In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import math

# Best Lattice Params from Optimization
# Params (dx, sy, rx, ry, sx): [0.8358, 0.7991, 0.4187, 0.4936, -0.2395]
LATTICE_DX = 0.8358098349066786
LATTICE_SY = 0.7990604144683316
LATTICE_RX = 0.4187365015402629
LATTICE_RY = 0.4935848002957603
LATTICE_SX = -0.23946249830796662

def get_tree_polygon(x=0, y=0, angle=0):
    trunk_w = 0.15
    trunk_h = 0.2
    base_w = 0.7
    mid_w = 0.4
    top_w = 0.25
    tip_y = 0.8
    tier_1_y = 0.5
    tier_2_y = 0.25
    base_y = 0.0
    trunk_bottom_y = -trunk_h
    
    coords = [
        (0.0, tip_y),
        (top_w/2, tier_1_y), (top_w/4, tier_1_y),
        (mid_w/2, tier_2_y), (mid_w/4, tier_2_y),
        (base_w/2, base_y),
        (trunk_w/2, base_y), (trunk_w/2, trunk_bottom_y),
        (-trunk_w/2, trunk_bottom_y), (-trunk_w/2, base_y),
        (-base_w/2, base_y),
        (-mid_w/4, tier_2_y), (-mid_w/2, tier_2_y),
        (-top_w/4, tier_1_y), (-top_w/2, tier_1_y)
    ]
    
    poly = Polygon(coords)
    if angle != 0:
        poly = affinity.rotate(poly, angle, origin=(0, 0))
    if x != 0 or y != 0:
        poly = affinity.translate(poly, xoff=x, yoff=y)
    return poly

def generate_lattice_solution(n):
    # Estimate grid size
    # We need N trees. Unit cell has 2 trees.
    # We need N/2 unit cells.
    # Grid side approx sqrt(N/2).
    # Let's generate a larger grid to be safe.
    side = int(math.sqrt(n)) + 4
    
    trees = []
    
    # Generate grid centered at 0
    for i in range(-side, side + 1):
        for j in range(-side, side + 1):
            # Lattice vectors
            # Pos = i * v1 + j * v2
            # v1 = (dx, 0)
            # v2 = (sx, sy)
            
            px = i * LATTICE_DX + j * LATTICE_SX
            py = i * 0          + j * LATTICE_SY
            
            # Tree 1 (0 deg)
            trees.append({
                'x': px,
                'y': py,
                'deg': 0.0,
                'poly': get_tree_polygon(px, py, 0.0)
            })
            
            # Tree 2 (180 deg)
            # Relative pos (rx, ry)
            p2x = px + LATTICE_RX
            p2y = py + LATTICE_RY
            trees.append({
                'x': p2x,
                'y': p2y,
                'deg': 180.0,
                'poly': get_tree_polygon(p2x, p2y, 180.0)
            })
            
    # Select N trees closest to center (Chebyshev distance for square shape)
    trees.sort(key=lambda t: max(abs(t['x']), abs(t['y'])))
    
    selected_trees = trees[:n]
    
    # Center the selected trees
    min_x = min(t['x'] for t in selected_trees)
    max_x = max(t['x'] for t in selected_trees)
    min_y = min(t['y'] for t in selected_trees)
    max_y = max(t['y'] for t in selected_trees)
    
    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    
    final_trees = []
    for i, t in enumerate(selected_trees):
        final_trees.append({
            'id': f"{n:03d}_{i}", # Placeholder ID, will fix later
            'x': t['x'] - center_x,
            'y': t['y'] - center_y,
            'deg': t['deg']
        })
        
    # Calculate score
    # Re-calculate bounds
    xs = [t['x'] for t in final_trees]
    ys = [t['y'] for t in final_trees]
    
    # Compute exact bounds using polygons
    min_bx, min_by, max_bx, max_by = float('inf'), float('inf'), float('-inf'), float('-inf')
    
    for t in final_trees:
        poly = get_tree_polygon(t['x'], t['y'], t['deg'])
        minx, miny, maxx, maxy = poly.bounds
        min_bx = min(min_bx, minx)
        min_by = min(min_by, miny)
        max_bx = max(max_bx, maxx)
        max_by = max(max_by, maxy)
        
    side_x = max_bx - min_bx
    side_y = max_by - min_by
    s = max(side_x, side_y)
    
    return final_trees, s

# Generate for all N
submission_rows = []
total_score = 0

for n in range(1, 201):
    trees, s = generate_lattice_solution(n)
    score = (s**2) / n
    total_score += score
    
    for i, t in enumerate(trees):
        # ID format: NNN_i
        row_id = f"{n:03d}_{i}"
        submission_rows.append({
            'id': row_id,
            'x': f"s{t['x']}",
            'y': f"s{t['y']}",
            'deg': f"s{t['deg']}"
        })

print(f"Estimated Total Score: {total_score}")

# Save
df = pd.DataFrame(submission_rows)
df.to_csv('submission.csv', index=False)


Estimated Total Score: 98.87588608546619
